In [ ]:
!pip install -qU google-genai langchain langchain_community langchain-google-community langchain-google-genai google-api-python-client google-auth google-auth-oauthlib


In [ ]:

import os
from google import genai
from google.genai import types
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from googleapiclient.discovery import build
from IPython.display import Markdown, display
from langchain.tools import tool
from langchain_google_community import GoogleSearchAPIWrapper


In [ ]:
# Set API key
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    GOOGLE_API_KEY = input("Please enter your Google API key: ")

if not GOOGLE_API_KEY:
    raise ValueError("Google API key not found.")

client = genai.Client(api_key=GOOGLE_API_KEY)
genai_model = "gemini-2.0-flash"

# Memory for conversation
conversation_history = []

# Google Search setup
os.environ["GOOGLE_CSE_ID"] = "Your cse id"
os.environ["GOOGLE_API_KEY"] = "your google api key"

search_wrapper = GoogleSearchAPIWrapper()

In [ ]:
# Load credentials from service account
SERVICE_ACCOUNT_FILE = "/content/inbound-dahlia-459707-m5-ed416da1fa41.json"
SCOPES = ["https://www.googleapis.com/auth/calendar"]

creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)
calendar_service = build("calendar", "v3", credentials=creds)
@tool
def calendar_tool(query: str) -> str:
    """Creates a calendar event (currently fixed time)."""
    event = {
        "summary": "AI Scheduled Meeting",
        "start": {"dateTime": "2025-05-16T10:00:00+05:00"},
        "end": {"dateTime": "2025-05-16T11:00:00+05:00"},
    }
    try:
        created_event = calendar_service.events().insert(calendarId="primary", body=event).execute()
        return f"Calendar event created: {created_event.get('htmlLink')}"
    except Exception as e:
        return f"Error creating calendar event: {e}"



In [ ]:
def decide_tool(query: str) -> str:
    prompt = f"""You are a smart router. Classify the query into one of these tools:
1. 'calendar' if it's about scheduling
2. 'search' for looking up information
3. 'chat' for general conversation

Query: "{query}"
Answer with one word: calendar, search, or chat."""

    decision = client.models.generate_content(
        model=genai_model,
        contents=[types.Content(role="user", parts=[types.Part(text=prompt)])]
    )
    return decision.candidates[0].content.parts[0].text.strip().lower()

In [ ]:
print("Chatbot with memory, search & calendar started! Type 'quit' to exit.")

while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        print("Goodbye!")
        break

    # Append user message to memory
    conversation_history.append(types.Content(role="user", parts=[types.Part(text=user_input)]))

    # Tool routing
    tool_choice = decide_tool(user_input)

    if tool_choice == "search":
        print("[🔍 Using Google Search]")
        search_result = search_wrapper.run(user_input)
        response_text = search_result
    elif tool_choice == "calendar":
        print("[📅 Using Google Calendar Tool]")
        response_text = calendar_tool.invoke(user_input)
    else:
        print("[💬 Chat Response]")
        response = client.models.generate_content(
            model=genai_model,
            contents=conversation_history
        )
        response_text = response.candidates[0].content.parts[0].text

    # Show and store model response
    display(Markdown(response_text))
    conversation_history.append(types.Content(role="model", parts=[types.Part(text=response_text)]))


In [ ]:
!pip install -qU google-genai langchain langchain_community langchain-google-community \
                langchain-google-genai google-api-python-client google-auth google-auth-oauthlib

In [ ]:
# Correct import for most stable versions of langchain_google_community
from langchain_google_community import GoogleSearchAPIWrapper


In [ ]:
# Install required packages
!pip install -qU google-genai langchain google-api-python-client google-auth google-auth-oauthlib

import os
from google import genai
from google.genai import types
from google.oauth2 import service_account
from googleapiclient.discovery import build
from langchain.tools import tool
from IPython.display import Markdown, display

# === User inputs for API keys and IDs ===
GOOGLE_API_KEY = input("Enter your Google API key: ").strip()
GOOGLE_CSE_ID = input("Enter your Google Custom Search Engine ID (CSE ID): ").strip()
SERVICE_ACCOUNT_FILE = "/content/inbound-dahlia-459707-m5-ed416da1fa41.json"  # Make sure you uploaded this file to Colab

if not os.path.exists(SERVICE_ACCOUNT_FILE):
    raise FileNotFoundError(f"{SERVICE_ACCOUNT_FILE} not found. Upload your Google service account JSON file.")

# === Setup Google GenAI Client ===
client = genai.Client(api_key=GOOGLE_API_KEY)
GENAI_MODEL = "gemini-2.0-flash"

# === Setup Google Calendar API client ===
SCOPES = ["https://www.googleapis.com/auth/calendar"]
creds = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
calendar_service = build("calendar", "v3", credentials=creds)

# === Google Search Tool (manual wrapper) ===
def google_search(query: str) -> str:
    from googleapiclient.discovery import build
    service = build("customsearch", "v1", developerKey=GOOGLE_API_KEY)
    try:
        res = service.cse().list(q=query, cx=GOOGLE_CSE_ID, num=3).execute()
        items = res.get("items", [])
        if not items:
            return "No results found."
        results_text = ""
        for i, item in enumerate(items, start=1):
            title = item.get("title")
            snippet = item.get("snippet")
            link = item.get("link")
            results_text += f"{i}. [{title}]({link})\n{snippet}\n\n"
        return results_text
    except Exception as e:
        return f"Error during search: {e}"

# === Calendar tool with natural language event creation (simple fixed example) ===
@tool
def calendar_tool(query: str) -> str:
    """
    Creates a calendar event based on natural language input.
    Currently uses a fixed example time.
    """
    try:
        # For demo purposes, you can extend this to parse date/time from query.
        event = {
            "summary": "AI Scheduled Meeting",
            "description": query,
            "start": {
                "dateTime": "2025-05-20T10:00:00+05:00",
                "timeZone": "Asia/Karachi",
            },
            "end": {
                "dateTime": "2025-05-20T11:00:00+05:00",
                "timeZone": "Asia/Karachi",
            },
        }
        created_event = calendar_service.events().insert(calendarId="primary", body=event).execute()
        return f"Calendar event created successfully: {created_event.get('htmlLink')}"
    except Exception as e:
        return f"Failed to create calendar event: {e}"

# === Decide which tool to use based on query content ===
def decide_tool(query: str) -> str:
    prompt = f"""You are a tool router. Choose one word from these options based on the user's query:
- 'calendar' for scheduling events
- 'search' for looking up information online
- 'chat' for general conversation

User query: "{query}"
Respond with only one word:"""

    decision = client.models.generate_content(
        model=GENAI_MODEL,
        contents=[types.Content(role="user", parts=[types.Part(text=prompt)])]
    )
    return decision.candidates[0].content.parts[0].text.strip().lower()

# === Chat memory ===
conversation_history = []

print("Chatbot with Google Search & Calendar integration started! Type 'quit' to exit.")

while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        print("Goodbye!")
        break

    conversation_history.append(types.Content(role="user", parts=[types.Part(text=user_input)]))

    tool_choice = decide_tool(user_input)
    print(f"[Using tool: {tool_choice}]")

    if tool_choice == "search":
        search_results = google_search(user_input)
        response_text = f"Here are some search results:\n\n{search_results}"
    elif tool_choice == "calendar":
        response_text = calendar_tool.invoke(user_input)
    else:
        # Just chat using Gemini without tools
        response = client.models.generate_content(
            model=GENAI_MODEL,
            contents=conversation_history
        )
        response_text = response.candidates[0].content.parts[0].text

    display(Markdown(response_text))
    conversation_history.append(types.Content(role="model", parts=[types.Part(text=response_text)]))


In [ ]:
# Install dependencies
!pip install -qU google-genai langchain google-api-python-client google-auth google-auth-oauthlib

import os
from google import genai
from google.genai import types
from google.oauth2 import service_account
from googleapiclient.discovery import build
from langchain.tools import tool
from IPython.display import Markdown, display

# User inputs
GOOGLE_API_KEY = input("Enter your Google API key: ").strip()
GOOGLE_CSE_ID = input("Enter your Google Custom Search Engine ID (CSE ID): ").strip()
SERVICE_ACCOUNT_FILE = "/content/inbound-dahlia-459707-m5-ed416da1fa41.json"  # Upload your service account JSON to Colab with this name

if not os.path.exists(SERVICE_ACCOUNT_FILE):
    raise FileNotFoundError(f"{SERVICE_ACCOUNT_FILE} not found. Please upload your Google service account JSON file.")

# Initialize Google GenAI client
client = genai.Client(api_key=GOOGLE_API_KEY)
GENAI_MODEL = "gemini-2.0-flash"

# Initialize Google Calendar API client
SCOPES = ["https://www.googleapis.com/auth/calendar"]
creds = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
calendar_service = build("calendar", "v3", credentials=creds)

# Google Search function that returns top results (title, snippet, link)
def google_search(query: str):
    service = build("customsearch", "v1", developerKey=GOOGLE_API_KEY)
    try:
        res = service.cse().list(q=query, cx=GOOGLE_CSE_ID, num=5).execute()
        items = res.get("items", [])
        if not items:
            return []
        results = []
        for item in items:
            results.append({
                "title": item.get("title"),
                "snippet": item.get("snippet"),
                "link": item.get("link"),
            })
        return results
    except Exception as e:
        return []

@tool
def calendar_tool(query: str) -> str:
    """
    Creates a calendar event based on the user's input.
    Currently uses a fixed time for demo purposes.
    """
    try:
        event = {
            "summary": "AI Scheduled Meeting",
            "description": query,
            "start": {
                "dateTime": "2025-05-20T10:00:00+05:00",
                "timeZone": "Asia/Karachi",
            },
            "end": {
                "dateTime": "2025-05-20T11:00:00+05:00",
                "timeZone": "Asia/Karachi",
            },
        }
        created_event = calendar_service.events().insert(calendarId="primary", body=event).execute()
        return f"Calendar event created successfully: {created_event.get('htmlLink')}"
    except Exception as e:
        return f"Failed to create calendar event: {e}"

def decide_tool(query: str) -> str:
    prompt = f"""You are a tool router. Choose one word from these options based on the user's query:
- 'calendar' for scheduling events
- 'search' for looking up information online
- 'chat' for general conversation

User query: "{query}"
Respond with only one word:"""

    decision = client.models.generate_content(
        model=GENAI_MODEL,
        contents=[types.Content(role="user", parts=[types.Part(text=prompt)])]
    )
    return decision.candidates[0].content.parts[0].text.strip().lower()

def generate_answer_with_search(user_question: str):
    search_results = google_search(user_question)
    if not search_results:
        return "I couldn't find relevant information through search."

    # Prepare context with search snippets + links
    context = "Here are some recent search results:\n"
    for i, result in enumerate(search_results, start=1):
        context += f"{i}. {result['title']}: {result['snippet']} (Source: {result['link']})\n\n"

    prompt = f"""You are a helpful assistant that answers questions using the following search results. Use the information to answer the question in detail and include source links.

Search Results:
{context}

Question: {user_question}

Answer:"""

    response = client.models.generate_content(
        model=GENAI_MODEL,
        contents=[types.Content(role="user", parts=[types.Part(text=prompt)])]
    )
    return response.candidates[0].content.parts[0].text.strip()

# Conversation memory to keep context for chat
conversation_history = []

print("Chatbot with search-grounded answers & calendar integration started! Type 'quit' to exit.")

while True:
    user_input = input("You: ").strip()
    if user_input.lower() == "quit":
        print("Goodbye!")
        break

    conversation_history.append(types.Content(role="user", parts=[types.Part(text=user_input)]))

    tool_choice = decide_tool(user_input)
    print(f"[Using tool: {tool_choice}]")

    if tool_choice == "search":
        # Generate grounded answer using search
        response_text = generate_answer_with_search(user_input)
    elif tool_choice == "calendar":
        response_text = calendar_tool.invoke(user_input)
    else:
        # Normal chat without tools, but with context memory
        response = client.models.generate_content(
            model=GENAI_MODEL,
            contents=conversation_history
        )
        response_text = response.candidates[0].content.parts[0].text

    display(Markdown(response_text))
    conversation_history.append(types.Content(role="model", parts=[types.Part(text=response_text)]))
